ODIR Data

In [12]:
#TRAINING SET ANNOTATION PROCESSING

import pandas as pd
df = pd.read_excel(r"Training Set\Annotation\training annotation (English).xlsx")
classes = ['N','D','G','C','A','H','M','O']
df[classes] = df[classes].astype(int)
df['labels'] = df[classes].values.tolist()
df[['ID', 'labels']].head()

,ID,labels
0,0,"[0, 0, 0, 1, 0, 0, 0, 0]"
1,1,"[1, 0, 0, 0, 0, 0, 0, 0]"
2,2,"[0, 1, 0, 0, 0, 0, 0, 1]"
3,3,"[0, 0, 0, 0, 0, 0, 0, 1]"
4,4,"[0, 1, 0, 0, 0, 0, 0, 1]"


In [ ]:
#ONSITE TEST SET ANNOTATION PROCESSING

import pandas as pd
df = pd.read_excel(r"On-site Test Set\Annotation\on-site test annotation (English).xlsx")
classes = ['N','D','G','C','A','H','M','O']
df[classes] = df[classes].astype(int)
df['labels'] = df[classes].values.tolist()
df[['ID', 'labels']].head()

,ID,labels
0,198,"[0, 0, 0, 1, 0, 1, 0, 0]"
1,663,"[0, 0, 0, 1, 0, 1, 0, 0]"
2,1025,"[0, 0, 0, 0, 0, 1, 0, 1]"
3,1139,"[0, 1, 0, 0, 0, 0, 0, 1]"
4,1141,"[0, 1, 0, 0, 0, 0, 0, 1]"


In [11]:
#OFFSITE TEST SET ANNOTATION PROCESSING

import pandas as pd
df = pd.read_excel(r"Off-site Test Set\Annotation\off-site test annotation (English).xlsx")
classes = ['N','D','G','C','A','H','M','O']
df[classes] = df[classes].astype(int)
df['labels'] = df[classes].values.tolist()
df[['ID', 'labels']].head()

,ID,labels
0,937,"[0, 1, 0, 0, 0, 1, 0, 0]"
1,967,"[0, 1, 0, 0, 0, 0, 0, 1]"
2,988,"[0, 1, 0, 0, 0, 0, 0, 1]"
3,995,"[0, 1, 0, 0, 0, 0, 0, 1]"
4,1000,"[0, 1, 0, 0, 0, 0, 0, 1]"


In [9]:
import cv2
import numpy as np

def apply_clahe(img):
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l)

    merged = cv2.merge((cl,a,b))
    return cv2.cvtColor(merged, cv2.COLOR_LAB2RGB)

In [10]:
import torchvision.transforms as T

mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

train_transform = T.Compose([
    T.Resize((224,224)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(15),
    T.ColorJitter(brightness=0.2, contrast=0.2),
    T.ToTensor(),
    T.Normalize(mean, std)
])

val_transform = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize(mean, std)
])

In [14]:
#TOTAL NUMBER OF IMAGES IN TRAINING, ON SITE AND OFF SITE TEST SETS

import os

image_dir_1 = r"Training Set\Images"
total_images_1 = len([f for f in os.listdir(image_dir_1) if f.endswith(('.jpg', '.png', '.jpeg'))])
print("total images in training set - ",total_images_1)

image_dir_2 = r"On-site Test Set\Images"
total_images_2 = len([f for f in os.listdir(image_dir_2) if f.endswith(('.jpg', '.png', '.jpeg'))])
print("total images in on-site test set - ",total_images_2)

image_dir_3 = r"Off-site Test Set\Images"
total_images_3 = len([f for f in os.listdir(image_dir_3) if f.endswith(('.jpg', '.png', '.jpeg'))])
print("total images in off-site test set - ",total_images_3)

total images in training set -  7000
total images in on-site test set -  2000
total images in off-site test set -  1000
